<a href="https://colab.research.google.com/github/ArturRSC/Telemetria_Automatica/blob/main/TelemetriaAutom%C3%A1tica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install ftfy
!pip install opencage
#importação das bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
from geopy.distance import geodesic
from opencage.geocoder import OpenCageGeocode
from google.colab import files
import time
# Fazer o upload do arquivo do excel
from google.colab import files
uploaded = files.upload()

Saving Report08-04.xlsx to Report08-04.xlsx


In [10]:
# Identificar o nome do arquivo carregado
file_name = list(uploaded.keys())[0]

# Carregar a planilha Excel
df = pd.read_excel(file_name)

# Exibir as primeiras linhas para verificar a estrutura
df.tail()

,Nome,Placa,Evento,Localização,Data,Velocidade,Latitude,Longitude,Altitude,Odômetro,Motorista,Empresa motorista
24641,NaN,SIT2B19,Entrada em zona,Rua Ibia (BR-262;MGC-262) & ~95 mts Rua Gerald...,2025-04-08 23:58:05,29,-19.938864,-43.476536,0,44014.0,HELBERT BATISTA,NaN
24642,NaN,SIR0G95,Reporte por tempo,Rua Aymorés - Rua José Augusto França & Rua Ar...,2025-04-08 23:58:11,0,-20.265381,-43.822906,0,42438.0,NaN,NaN
24643,NaN,RVW7J73-1,Reporte por tempo,Avenida Serafim Mota Barros - Rua Santana & Ru...,2025-04-08 23:58:13,0,-19.894514,-43.839622,0,56661.0,NaN,NaN
24644,NaN,TDV9A54,Reporte por tempo,Rua Desembargador Alonso Starling - Rua Soares...,2025-04-08 23:58:13,0,-19.942745,-44.017616,0,1270.0,NaN,NaN
24645,NaN,TDV9A60,Reporte por tempo,Rua Mestre Firmino - Rua Manoel Pinheiro Diniz...,2025-04-08 23:58:29,0,-19.911835,-44.073582,0,798.0,NaN,NaN


In [11]:
# Definir um dicionário de substituição para corrigir caracteres bugados
correcao_caracteres = {
    "Ã¡": "á", "Ã£": "ã", "Ã¢": "â", "Ãª": "ê", "Ã©": "é", "Ã¨": "è",
    "Ã­": "í", "Ã³": "ó", "Ãµ": "õ", "Ã´": "ô", "Ãº": "ú", "Ã§": "ç",
    "Ã€": "À", "Ã�": "Á", "Ã‚": "Â", "Ãƒ": "Ã", "Ã„": "Ä", "Ã…": "Å",
    "Ã‡": "Ç", "Ãˆ": "È", "Ã‰": "É", "ÃŠ": "Ê", "Ã‹": "Ë", "ÃŒ": "Ì",
    "Ã�": "Í", "ÃŽ": "Î", "Ã‘": "Ñ", "Ã’": "Ò", "Ã“": "Ó", "Ã”": "Ô",
    "Ã•": "Õ", "Ã–": "Ö", "Ã—": "×", "Ã˜": "Ø", "Ã™": "Ù", "Ãš": "Ú",
    "Ã›": "Û", "Ãœ": "Ü", "Ã�": "Ý", "ÃŸ": "ß", "Ã¡": "á", "Ã¢": "â",
    "Ã£": "ã", "Ã¤": "ä", "Ã¥": "å", "Ã¦": "æ", "Ã§": "ç", "Ã¨": "è",
    "Ã©": "é", "Ãª": "ê", "Ã«": "ë", "Ã¬": "ì", "Ã­": "í", "Ã®": "î",
    "Ã¯": "ï", "Ã°": "ð", "Ã±": "ñ", "Ã²": "ò", "Ã³": "ó", "Ã´": "ô",
    "Ãµ": "õ", "Ã¶": "ö", "Ã·": "÷", "Ã¸": "ø", "Ã¹": "ù", "Ãº": "ú",
    "Ã»": "û", "Ã¼": "ü", "Ã½": "ý", "Ã¾": "þ", "Ã¿": "ÿ"
}

def corrigir_texto(texto):
    """Corrige caracteres bugados substituindo por letras corretas."""
    if isinstance(texto, str):
        for bugado, correto in correcao_caracteres.items():
            texto = texto.replace(bugado, correto)
    return texto

# Aplicar a correção em todas as colunas do DataFrame
df = df.map(lambda x: corrigir_texto(x) if isinstance(x, str) else x)

# Aplicar a correção nos nomes das colunas
df.columns = [corrigir_texto(col) for col in df.columns]

# Exibir as primeiras linhas para verificar se os caracteres foram corrigidos corretamente
df.tail()

,Nome,Placa,Evento,Localização,Data,Velocidade,Latitude,Longitude,Altitude,Odômetro,Motorista,Empresa motorista
24641,NaN,SIT2B19,Entrada em zona,Rua Ibia (BR-262;MGC-262) & ~95 mts Rua Gerald...,2025-04-08 23:58:05,29,-19.938864,-43.476536,0,44014.0,HELBERT BATISTA,NaN
24642,NaN,SIR0G95,Reporte por tempo,Rua Aymorés - Rua José Augusto França & Rua Ar...,2025-04-08 23:58:11,0,-20.265381,-43.822906,0,42438.0,NaN,NaN
24643,NaN,RVW7J73-1,Reporte por tempo,Avenida Serafim Mota Barros - Rua Santana & Ru...,2025-04-08 23:58:13,0,-19.894514,-43.839622,0,56661.0,NaN,NaN
24644,NaN,TDV9A54,Reporte por tempo,Rua Desembargador Alonso Starling - Rua Soares...,2025-04-08 23:58:13,0,-19.942745,-44.017616,0,1270.0,NaN,NaN
24645,NaN,TDV9A60,Reporte por tempo,Rua Mestre Firmino - Rua Manoel Pinheiro Diniz...,2025-04-08 23:58:29,0,-19.911835,-44.073582,0,798.0,NaN,NaN


In [13]:
#Criar dataframe de acessos primários e secundários que podem ser visitados no dia: df_locais
dados_locais = {
    "Local": [
        "Galpão Chammas", "Laboratório Jardim Canadá", "Lote Jardim Canadá",
        "Mina Barão de Cocais - Cavas e Pilhas Canteiro Vale (ACESSO 1)",
        "Mina Barão de Cocais - Cavas e Pilhas Canteiro Vale (ACESSO 2)",
        "Mina Capanema (ACESSO 1)", "Mina Gongo Soco (ACESSO 1)",
        "Mina Gongo Soco (ACESSO 2)", "Ferrovia João Monlevade - EFVM / Vale (ACESSO 1)",
        "Mina Brucutu (ACESSO 1)", "Mina Brucutu - PDE Sul (ACESSO 1)", "Mina do Pico (ACESSO 1)",
        "Mina Paracatu (ACESSO 1)", "Mina Paracatu (ACESSO 2)", "Ferrovia Nova Era - EFVM / Vale (ACESSO 1)",
        "Ferrovia Nova Era - EFVM / Vale (FRENTE DE SERVIÇO)",
        "Mina Córrego do Feijão (ACESSO 1)", "Mina Córrego do Feijão (ACESSO 2)",
        "Mina Córrego do Feijão (ACESSO 3)", "Mina Córrego do Feijão (ACESSO 4)",
        "Barragem Germano (ACESSO 1)", "Santa Rita Durão (ACESSO 1)",
        "Mina Cuiabá - AngloGold (ACESSO 1)", "Mina Turmalina (ACESSO 1)",
        "EFVM | Integridade da Infraestrutura KM 465+360 (ACESSO 1)", "Mina Mar Azul (ACESSO 1)", "Mina Mar Azul (ACESSO 2)",
        "Miguel Burnier (ACESSO 1)", "Miguel Burnier (ACESSO 2)","Mina Cauê (ACESSO 1)", "Barragem de Itabiruçu (ACESSO 1)",
        "Barragem Conceição (ACESSO 1)", "Mina de Fábrica (ACESSO 1)"
    ],
    "Latitude": [
        -19.940563, -20.060605, -20.055630, -19.890156, -19.888303,
        -20.207931, -19.965451, -19.958197, -19.853518, -19.872053,
        -19.817795, -20.205191, -17.180762, -17.180624, -19.747774,
        -19.695215, -20.140347, -20.143405, -20.155327, -20.135059,
        -20.213197, -20.162302, -19.868426, -19.749354, -19.602845,
        -20.040788, -20.050653, -20.435645, -20.445380, -19.605003,
        -19.688499, -19.656930, -20.422648
    ],
    "Longitude": [
        -44.012557, -43.979885, -43.985511, -43.442593, -43.451588,
        -43.604647, -43.592801, -43.581132, -43.120937, -43.393233,
        -43.395049, -43.862075, -46.890102, -46.885328, -43.024028,
        -43.080339, -44.117570, -44.133863, -44.154478, -44.113841,
        -43.489024, -43.436597, -43.740635, -44.883803, -42.767673,
        -43.963142, -43.966111, -43.819066, -43.764838, -43.217101,
        -43.283502, -43.253601, -43.878055
    ],
    "Tipo": ["Secundário", "Secundário", "Secundário"] + ["Primário"] * 30
}

df_locais = pd.DataFrame(dados_locais)
print(df_locais)

                                                Local   Latitude  Longitude  \
0                                      Galpão Chammas -19.940563 -44.012557   
1                           Laboratório Jardim Canadá -20.060605 -43.979885   
2                                  Lote Jardim Canadá -20.055630 -43.985511   
3   Mina Barão de Cocais - Cavas e Pilhas Canteiro... -19.890156 -43.442593   
4   Mina Barão de Cocais - Cavas e Pilhas Canteiro... -19.888303 -43.451588   
5                            Mina Capanema (ACESSO 1) -20.207931 -43.604647   
6                          Mina Gongo Soco (ACESSO 1) -19.965451 -43.592801   
7                          Mina Gongo Soco (ACESSO 2) -19.958197 -43.581132   
8    Ferrovia João Monlevade - EFVM / Vale (ACESSO 1) -19.853518 -43.120937   
9                             Mina Brucutu (ACESSO 1) -19.872053 -43.393233   
10                  Mina Brucutu - PDE Sul (ACESSO 1) -19.817795 -43.395049   
11                            Mina do Pico (ACESSO 1

In [14]:
# Criar um dicionário para armazenar os dataframes de cada placa
df_por_placa = {}

# Identificar a coluna que contém as placas dos veículos
coluna_placa = "Placa"

# Criar um dataframe para cada placa
for placa in df[coluna_placa].unique():
    df_por_placa[placa] = df[df[coluna_placa] == placa].copy()

# Exibir as placas identificadas
df_por_placa.keys()

dict_keys(['RNT6B85', 'RUW6B08', 'SYJ8C60', 'SIB0C12-1', 'SYR2J73', 'RVF7F81', 'RNU4I64', 'SIB7B06', 'TDV9A54', 'RNT6B86', 'TDV9A61', 'RTE8A37', 'RTE4D15', 'RGB6H34', 'RUT6I73', 'RVW7J73-1', 'RUP8I09-1', 'RVW7J16', 'TDW1J44', 'RVW7J40-1', 'RVW7J84', 'TDU8I89', 'RTL6B60', 'SHX1F85', 'SHD7I83', 'RUF0E03', 'SYJ8C58', 'RVW7J99', 'SIO7F33', 'RTD4J63', 'RTE4D06', 'RTE4D04', 'SIT2B19', 'TDV9A60', 'RVU9F50', 'RNH0E24', 'TDV9A51', 'SIF7C74', 'TDU8I90', 'SIR0G95', 'TDU8I87', 'RFC7A10', 'RUU1I46', 'SYS3J90', 'TDV9A59', 'SYJ8C59', 'RTE4D01', 'TDV9A57', 'SHX1F91', 'RVC4D24', 'TDU3E72', 'SIF7D10', 'SYS3J89', 'SIC9H98', 'RUT6I71', 'SHV9B98', 'RNU4I60', 'SIX0C67', 'RTE4D07', 'TDV9A53', 'TDV9A58', 'TDV9A52', 'SIF8B66', 'SYR2J90', 'TDV9A56', 'RUA5H43', 'TDU8I88', 'RVW7J90', 'SYS3J92', 'SHQ4F51'])

In [33]:
coluna_data_hora = "Data"
coluna_evento = "Evento"
coluna_velocidade = "Velocidade"

# Converter a coluna de data para datetime
df[coluna_data_hora] = pd.to_datetime(df[coluna_data_hora], errors="coerce")

# Dicionário para armazenar os horários iniciais de uso
horarios_uso_por_placa = {}

for placa, df_veiculo in df_por_placa.items():
    df_veiculo = df_veiculo.sort_values(by=coluna_data_hora)

    # Encontrar o primeiro "Contato ON" e movimento subsequente
    contato_on = df_veiculo[df_veiculo[coluna_evento] == "Contato ON"]
    inicio_uso_veiculo = pd.NaT

    if not contato_on.empty:
        primeiro_contato_on = contato_on[coluna_data_hora].min()
        df_após_contato = df_veiculo[df_veiculo[coluna_data_hora] > primeiro_contato_on].copy()
        df_após_contato["velocidade_anterior"] = df_após_contato[coluna_velocidade].shift(1)
        df_inicio_movimento = df_após_contato[
            (df_após_contato["velocidade_anterior"].notna()) & (df_após_contato["velocidade_anterior"] < 10) & (df_após_contato[coluna_velocidade] >= 10)
        ]

        if not df_inicio_movimento.empty:
            inicio_uso_veiculo = df_inicio_movimento[coluna_data_hora].min()
        else:
            # Se não houve movimento > 10 após ligar, considera o próprio contato ON como início
            # (Pode precisar de ajuste dependendo da regra de negócio exata)
            inicio_uso_veiculo = primeiro_contato_on

    # Encontrar o último "Contato OFF" válido
    contatos_off = df_veiculo[df_veiculo[coluna_evento] == "Contato OFF"]
    fim_uso_veiculo = pd.NaT

    if not contatos_off.empty:
        # Ordenar contatos OFF do mais recente para o mais antigo para a lógica de verificação
        contatos_off_sorted = contatos_off.sort_values(by=coluna_data_hora, ascending=False)
        idx_list = list(contatos_off_sorted.index)

        fim_uso_veiculo = contatos_off_sorted.iloc[0][coluna_data_hora] # Assume o último OFF como padrão

        # Itera para encontrar um OFF precedido por movimento, começando do penúltimo
        for i in range(len(idx_list) - 1):
            idx_atual = idx_list[i]
            idx_anterior = idx_list[i + 1] # O OFF que veio antes cronologicamente

            contato_atual_dt = contatos_off_sorted.loc[idx_atual, coluna_data_hora]
            contato_anterior_dt = contatos_off_sorted.loc[idx_anterior, coluna_data_hora]

            df_entre_contatos = df_veiculo[
                (df_veiculo[coluna_data_hora] > contato_anterior_dt) &
                (df_veiculo[coluna_data_hora] < contato_atual_dt)
            ]

            # Verifica se houve movimento significativo *antes* deste Contato OFF
            if not df_entre_contatos.empty and (df_entre_contatos[coluna_velocidade] > 5).sum() >= 2:
                fim_uso_veiculo = contato_atual_dt # Encontrou o último OFF válido
                break # Para no primeiro que satisfaz a condição (que é o último cronológico)

        # Se o loop terminar sem break, fim_uso_veiculo já está com o último Contato OFF do dia

    # Armazena os resultados com os novos nomes
    horarios_uso_por_placa[placa] = {
        "Início de uso do veículo": inicio_uso_veiculo,
        "Fim de uso do veículo": fim_uso_veiculo
        }

# Exibe os resultados linha por linha
print("\nHorários de Início e Fim de Uso Calculados:")
for placa, horarios in horarios_uso_por_placa.items():
    inicio = horarios.get("Início de uso do veículo", "N/A")
    fim = horarios.get("Fim de uso do veículo", "N/A")
    print(f"Placa: {placa} | Início: {inicio} | Fim: {fim}")


Horários de Início e Fim de Uso Calculados:
Placa: RNT6B85 | Início: 2025-04-08 05:07:46 | Fim: 2025-04-08 18:21:35
Placa: RUW6B08 | Início: 2025-04-08 06:35:14 | Fim: 2025-04-08 16:20:09
Placa: SYJ8C60 | Início: NaT | Fim: NaT
Placa: SIB0C12-1 | Início: 2025-04-08 06:48:49 | Fim: 2025-04-08 18:13:21
Placa: SYR2J73 | Início: 2025-04-08 06:39:35 | Fim: 2025-04-08 18:39:07
Placa: RVF7F81 | Início: 2025-04-08 07:04:20 | Fim: 2025-04-08 16:51:45
Placa: RNU4I64 | Início: 2025-04-08 05:01:51 | Fim: 2025-04-08 18:50:24
Placa: SIB7B06 | Início: 2025-04-08 06:09:52 | Fim: 2025-04-08 17:05:31
Placa: TDV9A54 | Início: 2025-04-08 06:52:11 | Fim: 2025-04-08 14:59:05
Placa: RNT6B86 | Início: NaT | Fim: NaT
Placa: TDV9A61 | Início: 2025-04-08 06:32:53 | Fim: 2025-04-08 18:08:44
Placa: RTE8A37 | Início: 2025-04-08 07:20:28 | Fim: 2025-04-08 17:07:43
Placa: RTE4D15 | Início: 2025-04-08 06:33:44 | Fim: 2025-04-08 16:59:03
Placa: RGB6H34 | Início: 2025-04-08 07:28:04 | Fim: 2025-04-08 18:10:23
Placa: RU

In [31]:
# Definir parâmetros para o DBSCAN (mantido, embora não usado na lógica principal de minas)
raio_mina = 0.5  # Raio de 500 metros
min_amostras = 2  # Número mínimo de pontos no cluster

def encontrar_acesso(latitude, longitude, df_locais, raio_km=0.5):
    """Identifica o local de acesso mais próximo com classificação de tipo."""
    for _, local in df_locais.iterrows():
        distancia = geodesic((latitude, longitude), (local['Latitude'], local['Longitude'])).km
        if distancia <= raio_km:
            return local['Local'], local['Tipo']
    return None, None

# Dicionário para armazenar os horários finais por placa
horarios_mina_por_placa = {}

# Verificar quantas placas existem antes do loop
print("Total de placas:", len(df_por_placa))
print("Placas presentes:", list(df_por_placa.keys()))

for placa, df_veiculo in df_por_placa.items():
    print(f"\nProcessando placa: {placa}...")

    df_veiculo = df_veiculo.sort_values(by=coluna_data_hora)

    # Recuperar horários de uso previamente calculados
    inicio_uso_veiculo = horarios_uso_por_placa.get(placa, {}).get("Início de uso do veículo", pd.NaT)
    fim_uso_veiculo = horarios_uso_por_placa.get(placa, {}).get("Fim de uso do veículo", pd.NaT)

    coords = df_veiculo[['Latitude', 'Longitude']].dropna()

    if coords.empty:
        print(f"Placa {placa} sem coordenadas válidas!")
        # Salva com os nomes de coluna corretos
        horarios_mina_por_placa[placa] = {
            "Início de uso do veículo": inicio_uso_veiculo,
            "Chegada na Mina": pd.NaT, "Acesso Chegada Mina": None,
            "Saída da Mina": pd.NaT, "Acesso Saída Mina": None,
            "Fim de uso do veículo": fim_uso_veiculo,
            "Observação": "Sem coordenadas válidas"
        }
        continue

    # Aplica DBSCAN
    clustering = DBSCAN(eps=raio_mina / 6371, min_samples=min_amostras, metric='haversine').fit(np.radians(coords))
    df_veiculo['Cluster'] = -1
    df_veiculo.loc[coords.index, 'Cluster'] = clustering.labels_

    # Identificar locais de acesso
    df_veiculo[['Local_Acesso', 'Tipo_Acesso']] = df_veiculo.apply(
        lambda row: encontrar_acesso(row['Latitude'], row['Longitude'], df_locais, raio_mina),
        axis=1, result_type='expand'
    )

    df_mina_primaria = df_veiculo[df_veiculo['Tipo_Acesso'] == 'Primário'].dropna(subset=['Local_Acesso'])
    df_mina_secundaria = df_veiculo[df_veiculo['Tipo_Acesso'] == 'Secundário'].dropna(subset=['Local_Acesso'])

    # Inicializar variáveis
    chegada_mina = pd.NaT
    saida_mina = pd.NaT
    acesso_chegada = None
    acesso_saida = None
    periodos_minas = []
    observacao_list = []

    # Processar acessos primários
    if not df_mina_primaria.empty:
        df_mina_primaria = df_mina_primaria.sort_values(coluna_data_hora)
        df_mina_primaria['Mudanca'] = (df_mina_primaria['Local_Acesso'] != df_mina_primaria['Local_Acesso'].shift(1)).cumsum()
        grupos = df_mina_primaria.groupby('Mudanca')
        for _, grupo in grupos:
            if not grupo.empty:
                periodo = {
                    'local': grupo['Local_Acesso'].iloc[0],
                    'inicio': grupo[coluna_data_hora].min(),
                    'fim': grupo[coluna_data_hora].max(),
                    'duracao': (grupo[coluna_data_hora].max() - grupo[coluna_data_hora].min()).total_seconds() / 60
                }
                periodos_minas.append(periodo)

    # Condição 4: Determinar mina principal (>= 30 min)
    minas_passagem_curta = []
    outras_minas_longas = []
    principal = None
    if periodos_minas:
        periodos_longos = [p for p in periodos_minas if p['duracao'] >= 30]
        periodos_curtos = [p for p in periodos_minas if p['duracao'] < 30]
        minas_passagem_curta = [p['local'] for p in periodos_curtos]
        if periodos_longos:
            principal = max(periodos_longos, key=lambda x: x['duracao'])
            chegada_mina = principal['inicio']
            saida_mina = principal['fim']
            acesso_chegada = principal['local']
            acesso_saida = principal['local']
            outras_minas_longas = [p['local'] for p in periodos_longos if p['local'] != principal['local']]

    if outras_minas_longas:
        observacao_list.append(f"Passou por (>= 30 min): {', '.join(list(set(outras_minas_longas)))}")
    if minas_passagem_curta:
        observacao_list.append(f"Passou por (< 30 min): {', '.join(list(set(minas_passagem_curta)))}")

    # Condição 5: Chegada na mina antes das 2:00 AM
    acordou_na_mina_flag = False
    if not pd.isna(chegada_mina) and chegada_mina.hour < 2:
        obs_acordou_mina = f"Acordou na mina ({acesso_chegada})"
        if obs_acordou_mina not in observacao_list:
            observacao_list.append(obs_acordou_mina)
            acordou_na_mina_flag = True
        # Limpar horários
        inicio_uso_veiculo = pd.NaT
        chegada_mina = pd.NaT
        acesso_chegada = None

    # Processar locais secundários
    locais_secundarios = df_mina_secundaria['Local_Acesso'].unique()
    observacao_secundarios = []
    if len(locais_secundarios) > 0:
        observacao_secundarios = ["Passou por " + ", ".join(locais_secundarios)]

    # Lógica "Iniciou dia na mina"
    if not acordou_na_mina_flag and not df_mina_primaria.empty:
         primeiro_registro_hora = df_veiculo[coluna_data_hora].min()
         # Verifica se o primeiro registro existe antes de tentar acessá-lo
         primeiro_registro = df_veiculo.loc[df_veiculo[coluna_data_hora] == primeiro_registro_hora]
         if not primeiro_registro.empty:
             primeiro_registro_local = primeiro_registro['Local_Acesso'].iloc[0]
             primeiro_registro_tipo = primeiro_registro['Tipo_Acesso'].iloc[0]
             if primeiro_registro_tipo == 'Primário':
                 obs_dormiu_mina = f"Caminhão iniciou dia na mina ({primeiro_registro_local})"
                 if obs_dormiu_mina not in observacao_list:
                     observacao_list.append(obs_dormiu_mina)
                 inicio_uso_veiculo = pd.NaT

    # Lógica "Não rodou no dia"
    odometro_vals = df_veiculo['Odômetro'].dropna()
    nao_rodou = False
    if not odometro_vals.empty and len(odometro_vals) > 1: # Precisa de pelo menos 2 valores para calcular diferença
        if (odometro_vals.max() - odometro_vals.min()) < 10:
            observacao_list.append("Não rodou no dia")
            nao_rodou = True
            if len(locais_secundarios) > 0:
                 observacao_secundarios = ["Ficou em " + ", ".join(locais_secundarios)]
            else:
                 locais_primarios_visitados = df_mina_primaria['Local_Acesso'].unique()
                 if len(locais_primarios_visitados) >= 1: # Se estava em alguma mina primária
                      # Verifica se ficou apenas em UMA mina primária
                      if len(locais_primarios_visitados) == 1:
                           observacao_list.append(f"Ficou em {locais_primarios_visitados[0]}")
                      # Se ficou em mais de uma (improvável sem rodar), adiciona nota genérica se necessário
                      # else: observacao_list.append("Ficou em locais primários")

            # Limpar todos os horários se não rodou
            inicio_uso_veiculo = pd.NaT
            chegada_mina = pd.NaT
            saida_mina = pd.NaT
            fim_uso_veiculo = pd.NaT
            acesso_chegada = None
            acesso_saida = None
    elif len(odometro_vals) <= 1:
        # Se tem 0 ou 1 leitura de odômetro, não pode calcular diferença, assume que não rodou
        # Pode ser útil adicionar uma observação específica aqui se necessário
        if "Não rodou no dia" not in observacao_list: # Evita duplicar se já foi marcado por outra razão
             observacao_list.append("Não rodou no dia (Odômetro insuficiente)")
             nao_rodou = True
             # Limpar horários também neste caso
             inicio_uso_veiculo = pd.NaT
             chegada_mina = pd.NaT
             saida_mina = pd.NaT
             fim_uso_veiculo = pd.NaT
             acesso_chegada = None
             acesso_saida = None
             # Lógica de "Ficou em" pode ser aplicada aqui também se desejar

    # Combinar observações secundárias
    if observacao_secundarios:
        if nao_rodou:
            observacao_list = [obs for obs in observacao_list if not obs.startswith("Passou por")]
        observacao_list.extend(observacao_secundarios)

    # Formatar observação final
    observacao = " | ".join(filter(None, list(set(observacao_list)))) if observacao_list else None # Usar set para remover duplicatas exatas

    # Salvar resultados com os NOMES DE COLUNA CORRETOS
    horarios_mina_por_placa[placa] = {
        "Início de uso do veículo": inicio_uso_veiculo,
        "Chegada na Mina": chegada_mina,
        "Acesso Chegada Mina": acesso_chegada,
        "Saída da Mina": saida_mina,
        "Acesso Saída Mina": acesso_saida,
        "Fim de uso do veículo": fim_uso_veiculo,
        "Observação": observacao
    }

print("\nResultado Final:")
print(horarios_mina_por_placa)

Total de placas: 70
Placas presentes: ['RNT6B85', 'RUW6B08', 'SYJ8C60', 'SIB0C12-1', 'SYR2J73', 'RVF7F81', 'RNU4I64', 'SIB7B06', 'TDV9A54', 'RNT6B86', 'TDV9A61', 'RTE8A37', 'RTE4D15', 'RGB6H34', 'RUT6I73', 'RVW7J73-1', 'RUP8I09-1', 'RVW7J16', 'TDW1J44', 'RVW7J40-1', 'RVW7J84', 'TDU8I89', 'RTL6B60', 'SHX1F85', 'SHD7I83', 'RUF0E03', 'SYJ8C58', 'RVW7J99', 'SIO7F33', 'RTD4J63', 'RTE4D06', 'RTE4D04', 'SIT2B19', 'TDV9A60', 'RVU9F50', 'RNH0E24', 'TDV9A51', 'SIF7C74', 'TDU8I90', 'SIR0G95', 'TDU8I87', 'RFC7A10', 'RUU1I46', 'SYS3J90', 'TDV9A59', 'SYJ8C59', 'RTE4D01', 'TDV9A57', 'SHX1F91', 'RVC4D24', 'TDU3E72', 'SIF7D10', 'SYS3J89', 'SIC9H98', 'RUT6I71', 'SHV9B98', 'RNU4I60', 'SIX0C67', 'RTE4D07', 'TDV9A53', 'TDV9A58', 'TDV9A52', 'SIF8B66', 'SYR2J90', 'TDV9A56', 'RUA5H43', 'TDU8I88', 'RVW7J90', 'SYS3J92', 'SHQ4F51']

Processando placa: RNT6B85...

Processando placa: RUW6B08...

Processando placa: SYJ8C60...

Processando placa: SIB0C12-1...

Processando placa: SYR2J73...

Processando placa: RVF7F8

In [32]:
from google.colab import files

# O DataFrame usará as chaves do dicionário como nomes de coluna
df_resultados = pd.DataFrame(horarios_mina_por_placa).T

# Salvar o CSV no ambiente temporário do Colab
arquivo_csv = "resultados_telemetria.csv"
df_resultados.to_csv(arquivo_csv, encoding="utf-8", sep=";", index_label="Placa")

# Baixar o arquivo
files.download(arquivo_csv)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>